In [4]:
l = [-100, 20, 30, 50, 76, 100, 200]
x = 30

for num in l:
    if num == x:
        print(True)
        break
else:
    print(False)

True


In [2]:
from math import log2

In [3]:
log2(10_000_000)

23.253496664211536

In [6]:
%%time
x = list(range(10_000_000))
x_set = set(x)

CPU times: user 583 ms, sys: 328 ms, total: 911 ms
Wall time: 931 ms


In [7]:
%%time
20_000_000 in x

CPU times: user 149 ms, sys: 1.52 ms, total: 151 ms
Wall time: 155 ms


False

In [8]:
%%time
20_000_000 in x_set

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.01 µs


False

In [ ]:
l = [-100, 20, 30, 50, 76, 100, 200]
x = 30

x in l  # bad O(N)
x in set(l)  # good O(1)

In [9]:
hash((1, 2, 3))

529344067295497451

In [10]:
hash((1, 2, 4))

-4363729961677198915

In [11]:
hash([1, 2, 3])

TypeError: unhashable type: 'list'

In [12]:
x = {1, 2, [3, 4]}

TypeError: unhashable type: 'list'

In [15]:
{
    (1, 2): 5,
    5: [1, 2, 3]
}

{(1, 2): 5, 5: [1, 2, 3]}

# Анализ данных на Python

### Семинар 5. Множества и словари

# Что такое Хэш-таблица? 

Вы - продавец в магазине. Когда покупатель что-то у вас покупает, вы проверяете стоимость товара по книге. 

In [ ]:
book = [('яйца', 60), ('чай', 16), ('кофе', 35), ('лён', 20), 
        ('петрушка', 15), ('торт', 10), ('арбуз', 60), ('йогурт', 35), 
        ('соя', 20), ('ролтон', 42), ('бобы', 10), ('глаз дракона', 2)]

Как найти сколько стоят бобы? Листать книгу, читать в ней каждую строчку до тех пор пока мы не найдём ответ. 

In [ ]:
x = 'бобы'

for item in book:
    if item[0] == x:
        print(item[1])

__Вопрос:__ Если у нас всего $n$ продуктов, сколько действий нам надо будет сделать в худшем случае? 

Как-то долговато. Давайте попробуем ускориться. Одной из идей ускорения может быть сортировка. Если отсортировать все продукты по их названию, искать будет легче. 

In [ ]:
book = sorted(book, key=lambda w: w[0])
book

Будем открывать книгу в середине. Там буква "п". Нам нужна буква "б", она левее буквы "п". Откроем серидну левой части книги, там буква "й", нам нужно еще левее, снова откроем середину. Будем так делать до тех пор, пока не найдём бобы. Такая процедура будет работать быстрее, она называется __бинарный поиск.__

In [ ]:
# Попытайтесь на досуге написать такой поиск самостоятельно :) 

    # ┬─┬ ノ( ゜-゜ノ)       Писать код самому? 
    
    # (╯° □°)╯︵ ┻━┻ 

__Вопрос:__ Если у нас всего $n$ продуктов, сколько действий нам надо будет сделать в худшем случае? 

Всё ещё долго. А можно ещё быстрее? Конечно можно. Давайте наймём помощницу по имени Алиса и заставим её выучить всю книгу с продуктами и ценами наизусть. Тогда мы сможем задавать ей вопрос и моментально будем получать ответ на него. Просто чудо, а не помощница! Где бы взять такую... 

Попробуем создать её из тех структур данных, которые мы уже знаем. А именно: из массивов. Для этого будем использовать __хеш-функцию.__ Хеш-функция - это такая функция, которая на вход получает строку и возвращает число. Они и поможет нам создать свою Алису.

In [ ]:
x = [0]*33 # заведём пустой массив
x[:10]

Пусть наша хэш-функция возвращает номер первой буквы слова в алфавите. 

In [1]:
def simple_hash(x):
    alph = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
    return alph.index(x[0])

simple_hash('яйца')

32

Положим в массив $x$ на $32$ позицию цену на яйца. По аналогии сделаем со всеми продуктами и их ценами. 

In [ ]:
for food, price in book:
    x[simple_hash(food)] = price
    
x

Хэщ-функция в нашем примере связывает каждое название с одним индексом. На месте этого индекса в векторе $x$ и лежит нужная цена. __Поздравляю, мы создали свою Алису!__

А теперь к нам приходит клиент и спрашивает: "А сколько стоит торт?" Мы легко можем ответить на его вопрос: 

In [ ]:
ind = simple_hash('торт')
x[ind]

И мы делаем это моментально, без перебора как в первых двух случаях. 

__Вопросы:__

- Понятное дело, что на практике хеш-функции устроены сложнее, у той функции, которую мы тут использовали есть куча проблем: какие это проблемы? 
- Как можно попытаться решить эти проблемы? 

В python хэш-таблицы реализованы в виде словарей и множеств. Давайте с ними познакомимся. 

# Множества (set)

Мы уже знаем списки и кортежи - упорядоченные структуры, которые могут хранить в себе объекты любых типов, к которым мы можем обратиться по индексу. Теперь поговорим о стуктурах неупорядоченных - множествах и словарях. 

Множества хранят некоторое количество объектов, но, в отличие от списка, один объект может храниться в множестве не более одного раза. Кроме того, порядок элементов множества произволен, им нельзя управлять.

Тип называется set, это же является конструктором типа, т.е. в функцию set можно передать произвольную последовательность, и из этой последовательности будет построено множество:

In [1]:
print(set([10, 20, 30])) # передаем список
print(set((4, 5, 6))) # передаем tuple
print(set(range(10))) # послед-сть от 0 до 10 не включая
print(set()) # пустое множество


Другой способ создать множество - это перечислить его элементы в фигурных скобках (список - в квадратных, кортеж в круглых, а множество - в фигурных)

In [2]:
primes = {2, 3, 5, 7}
animals = {"cat", "dog", "horse", 'cat'}
print(primes)
print(animals)

Кстати, обратите внимание, что множество может состоять только из уникальных объектов. Выше множество animals включает в себя только одну кошку несмотря на то, что в конструктор мы передали 'cat' два раза. Преобразовать список в множество - самый простой способ узнать количество уникальных объектов.

Со множествами работает почти всё, что работает с последовательностями (но не работают индексы, потому что элементы не хранятся упорядоченно).

In [3]:
print(len(primes)) # длина
print(11 in primes) # проверка на наличие элемента in хорошо и быстро работает для множеств
print("cow" in animals)

Все возможные операции с множествами: https://docs.python.org/3/library/stdtypes.html#set-types-set-frozenset

Отдельно мы посмотрим на так называемые операции над множествами. Если вы знаете круги Эйлера, то помните как различают объекты множеств - пересечение, объекты, которые принадлежат множеству а, но не принадлежат b и так далее. Давайте посмотрим, как эти операции реализованы в питоне.

In [4]:
a = {1, 2, 3, 4}
b = {3, 4, 5, 6}
c = {2, 3}

print(c <= a) # проверка на подмножество (с подномжество a)
print(c <= b) # не подмножество, т.к. в b нет 2 
print(a >= c) # а надмножество с
print(a | b) # объединение
print(a & b) # пересечение
print(a - b) # разность множеств (все что в a, кроме b)
print(a ^ b) # симметрическая разность множеств (объединение без пересечения)

c = a.copy() # копирование множества, или set(a)
print(c)

Предыдущие операции не меняли множества, создавали новые. А как менять множество:


In [5]:
s = {1, 2, 3}
s.add(10) # добавить
print(s) # обратите внимание, что порядок элементов непредсказуем
s.remove(1) # удаление элемента
s.discard(1) # аналогично, но не будет ошибки, если вдруг такого элемента нет в множестве
print(s)
x = s.pop() # удаляет и возвращает один произвольный элемент множества (можем сохранить его в переменную)
print(s)
print(x)
s.clear() # очистить
print(s)

Как мы сокращали арифметические операции раньше (например, +=), так же можно сокращать операции над множествами.

In [6]:
s |= {10, 20} # s = s | {10, 20} # объединение множества s с {10,20}
print(s)
# s ^=, s &= и т.п.

### Задача 1 (на самом деле очень простая)

Дан список чисел, который может содержать до 100000 чисел. Определите, сколько в нем встречается различных чисел.

#### Пример 1

##### Ввод	

1 2 3 2 1

##### Вывод

3

#### Пример 2
##### Ввод	

1 2 3 4 5 1 2 1 2 7 3

##### Вывод
6

In [7]:
# решение здесь


### Задача 2 (факультативы)
Группа из 3 студентов пишет заявки на желаемые факультативы из списка: английский, немецкий, право, математика, сольфеджио. Факультатив откроют, если на него запишутся все студенты. Каждый студент может выбрать минимум один и максимум три факультатива. Нужно посчитать количество факультативов, которые откроются.

*Пример*

**Ввод:**

английский сольфеджио право  
математика сольфеджио  
немецкий право

**Вывод:**

0

**Ввод:**

математика немецкий право  
математика немецкий  
немецкий право математика 

**Вывод:**  
2

In [8]:
# решение здесь


Давайте модифицируем эту задачу, и представим, что только два студента выбирают факультативы, откроются факультативы, которые выбрал, хотя бы один студент. Но затем выясняется, что один из факультативов из списка отменили. Выводить будет через запятую те факультативы, которые откроются.

*Пример*  
**Ввод:**

математика немецкий право  
математика английский  
право

**Вывод:**  
математика, немецкий, английский

In [9]:
# решение здесь


## Задача 3: [камешки](https://leetcode.com/problems/jewels-and-stones/)

У Дори в глубинах океана есть кучка камней. Часть камней из этой кучки драгоценные. Недавно она пересчитала все драгоценные и забыла сколько их. Чтобы больше не забывать, Дори решила написать на питоне функцию, которая будет считать камни за неё.

Напишите на python функцию, которая принимает на вход список драгоценных камней $J$ и список камней, которые есть у Дори $S$. На выход функция возвращает число драгоценных камней в запасах Дори.

__Примеры:__ 

> Input: J = "aA", S = "aAAbbbb" <br />
Output: 3

Тут драгоценными считаются камни a и A. У Дори есть камни aAAbbbb. Среди них три драгоценных, aAA.

>Input: J = "z", S = "ZZ" <br />
Output: 0

Драгоценными мы считаем только камень z. У Дори два камня, оба обычные.

<img src="https://steemitimages.com/0x0/https://media.makeameme.org/created/repeat-repeat-repeat-5984a6.jpg" height="400" width="400">

In [10]:
def numJewelsInStones(J, S):
    
    ### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
    # will the code be with you
    
    return

In [11]:
def test_problem_13(func, test_data):
    for inputs, true_answer in test_data:
        answer = func(*inputs)
        assert answer == true_answer, f'Expected {true_answer}, got {answer}. Input: {inputs}'
    print("OK!")

In [12]:
NUM_JEWELS_IN_STONES_TESTS_DATA = [
        (("aA", "aAAbbbb"), 3),
        (("z","ZZ"),0)
    ]
test_problem_13(numJewelsInStones, NUM_JEWELS_IN_STONES_TESTS_DATA)

AssertionError: Expected 3, got None. Input: ('aA', 'aAAbbbb')

__Пара слов об эффективности:__

In [13]:
from random import random
n_obs = 10**6

In [14]:
mylist = [random() for _ in range(n_obs)]
myset = set(mylist)

In [15]:
%%timeit
0.5 in mylist  # список

11 ms ± 1.6 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
%%timeit
0.5 in myset   # множество

56.7 ns ± 0.149 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


# Словари (dict)
Обычный массив (в питоне это список) можно понимать как функцию, которая сопоставляет начальному отрезку натурального ряда какие-то значения.

Давайте посмотрим на списки непривычным способом. Списки - это функции (отображения), которые отображают начальный ряд натуральных чисел в объекты (проще говоря - преводят число 0,1,2,3... во что-то): 

In [17]:
l = [10, 20, 30, 'a']
print(l[0])
print(l[1])
print(l[2])
print(l[3])

10
20
30
a


В словарях отображать можно не только начала натурального ряда, а произвольные объекты. Представьте себе настоящий словарь или телефонную книжку. Имени человека соответствует номер телефона.

Классическое использование словарей в анализе данных: хранить частоту слова в тексте.

кот $\rightarrow$ 10

и $\rightarrow$ 100

Тейлора $\rightarrow$ 2

Словарь состоит из набора ключей и соответствующих им значений. Значения могут быть любыми объектами (также как и в списке, хранить можно произвольные объекты). А ключи могут быть почти любыми объектами, но только неизменяемыми. В частности числами, строками, кортежами. Список или множество не могут быть ключом.

Одному ключу соответствует ровно одно значение. Но одно и то же значение, в принципе, можно сопоставить разным ключам.

In [18]:
a = dict()
a[(2,3)] = [2,3] # кортеж может быть ключом, потому что он неизменямый
a

{(2, 3): [2, 3]}

In [19]:
b = dict()
b[[2,3]] = [2,3] # а список уже нет, получим ошибку
print(b)

TypeError: unhashable type: 'list'

### Создание словаря
В фигурных скобках (как множество), через двоеточие ключ:значение

In [20]:
d1 = {"кот": 10, "и": 100, "Тейлора": 2}
print(d1)

{'кот': 10, 'и': 100, 'Тейлора': 2}


Через функцию dict(). Обратите внимание, что тогда ключ-значение задаются не через двоеточие, а через знак присваивания. А строковые ключи пишем без кавычек - по сути мы создаем переменные с такими названиями и присваиваим им значения (а потом функция dict() уже превратит их в строки).

In [21]:
d2 = dict(кот=10, и=100, Тейлора=2)
print(d2) # получили тот же результат, что выше

{'кот': 10, 'и': 100, 'Тейлора': 2}


И третий способ - передаем функции dict() список списков или кортежей с парами ключ-значение.

In [22]:
d3 = dict([("кот", 10), ("и", 100), ("Тейлора", 2)]) # перечисление (например, список) tuple
print(d3)

{'кот': 10, 'и': 100, 'Тейлора': 2}


Помните, когда мы говорили про списки, мы обсуждали проблему того, что важно создавать именно копию объекта, чтобы сохранять исходный список. Копию словаря можно сделать так

In [23]:
d4 = dict(d3) # фактически, копируем dict который строчкой выше
print(d4)

{'кот': 10, 'и': 100, 'Тейлора': 2}


In [24]:
d1 == d2 == d3 == d4 # Содержание всех словарей одинаковое

True

Пустой словарь можно создать двумя способами.

In [25]:
d2 = {} # это пустой словарь (но не пустое множество)
d4 = dict()
print(d2, d4)

{} {}


### Операции со словарями

Как мы уже говорили, словари неупорядоченные структуры и обратиться по индексу к объекту уже больше не удастся.

In [26]:
d1[1] # выдаст ошибку во всех случах кроме того, если в вашем словаре вдруг есть ключ 1

KeyError: 1

Но можно обращаться к значению по ключу.

In [28]:
print(d1['кот'])

10


Можно создать новую пару ключ-значение. Для этого просто указываем в квадратных скобках название нового ключа.

In [29]:
d1[1] = 'test'
print(d1[1]) # теперь работает!

test


Внимание: если элемент с указанным ключом уже существует, новый с таким же ключом не добавится! Ключ – это уникальный идентификатор элемента. Если мы добавим в словарь новый элемент с уже существующим ключом, мы просто изменим старый – словари являются изменяемыми объектами. 

In [30]:
d1["кот"] = 11 # так же как в списке по индексу - можно присвоить новое значение по ключу
print(d1['кот'])
d1["кот"] += 1 # или даже изменить его за счет арифметической операции
print(d1['кот'])

11
12


А вот одинаковые значения в словаре могут быть.

In [31]:
d1['собака'] = 12
print(d1)

{'кот': 12, 'и': 100, 'Тейлора': 2, 1: 'test', 'собака': 12}


Кроме обращения по ключу, можно достать значение с помощью метода .get(). Отличие работы метода в том, что если ключа еще нет в словаре, он не генерирует ошибку, а возвращает объект типа None ("ничего"). Это очень полезно в решении некоторых задач. 

In [32]:
print(d1.get("кот")) # вернул значение 11
print(d1.get("ктоо")) # вернут None
print(d1['ктоо']) # ошибка

12
None


KeyError: 'ктоо'

Удобство метода .get() заключается в том, что мы сами можем установить, какое значение будет возвращено, в случае, если пары с выбранным ключом нет в словаре. Так, вместо None мы можем вернуть строку Not found, и ломаться ничего не будет:

In [36]:
print(d1.get("ктоо", 'Not found')) # передаем вторым аргументом, что возвращать
print(d1.get("ктоо", False)) # передаем вторым аргументом, что возвращать

Not found
False


Также со словарями работают уже знакомые нам операции - проверка количества элементов, проверка на наличие объектов.

In [37]:
print(d1)
print("кот" in d1) # проверка на наличие ключа
print("ктоо" not in d1) # проверка на отстуствие ключа


{'кот': 12, 'и': 100, 'Тейлора': 2, 1: 'test', 'собака': 12}
True
True


Удалить отдельный ключ или же очистить весь словарь можно специальными операциями.

In [38]:
del d1["кот"] # удалить ключ со своим значением
print(d1)
d1.clear() # удалить все
print(d1)

{'и': 100, 'Тейлора': 2, 1: 'test', 'собака': 12}
{}


У словарей есть три метода, с помощью которых мы можем сгенерировать список только ключей, только значений и список пар ключ-значения (на самом деле там несколько другая структура, но ведет себя она очень похоже на список).

In [39]:
print(d1.values()) # только значения
print(d1.keys()) # только ключи
print(d1.items()) # только значения

dict_values([])
dict_keys([])
dict_items([])


## Пример

In [40]:
product = ['яйца', 'чай', 'кофе', 'банан', 'петрушка', 'сода', 
           'яблочко', 'йогурт', 'соя', 'беозар', 'бобы', 'печень дракона']

price = [60, 16, 35, 20, 15, 10, 60, 35, 20, 42, 10, 2]

zip(product, price)

In [41]:
book_dict = dict(zip(product, price))
book_dict

{'яйца': 60,
 'чай': 16,
 'кофе': 35,
 'банан': 20,
 'петрушка': 15,
 'сода': 10,
 'яблочко': 60,
 'йогурт': 35,
 'соя': 20,
 'беозар': 42,
 'бобы': 10,
 'печень дракона': 2}

In [42]:
book_dict.keys()

dict_keys(['яйца', 'чай', 'кофе', 'банан', 'петрушка', 'сода', 'яблочко', 'йогурт', 'соя', 'беозар', 'бобы', 'печень дракона'])

In [43]:
book_dict.values()

dict_values([60, 16, 35, 20, 15, 10, 60, 35, 20, 42, 10, 2])

In [44]:
book_dict.items()

dict_items([('яйца', 60), ('чай', 16), ('кофе', 35), ('банан', 20), ('петрушка', 15), ('сода', 10), ('яблочко', 60), ('йогурт', 35), ('соя', 20), ('беозар', 42), ('бобы', 10), ('печень дракона', 2)])

In [45]:
for k, v in book_dict.items():
    print(f"Продукт {k} стоит {v} рублей")

Продукт яйца стоит 60 рублей
Продукт чай стоит 16 рублей
Продукт кофе стоит 35 рублей
Продукт банан стоит 20 рублей
Продукт петрушка стоит 15 рублей
Продукт сода стоит 10 рублей
Продукт яблочко стоит 60 рублей
Продукт йогурт стоит 35 рублей
Продукт соя стоит 20 рублей
Продукт беозар стоит 42 рублей
Продукт бобы стоит 10 рублей
Продукт печень дракона стоит 2 рублей


In [46]:
{i: i**3 for i in range(2, 20, 3)} # словарь 

{2: 8, 5: 125, 8: 512, 11: 1331, 14: 2744, 17: 4913}

Ну, и раз уж питоновские словари так похожи на обычные, давайте представим, что у нас есть словарь, где все слова многозначные. Ключом будет слово, а значением ‒ целый список. 

In [47]:
my_dict = {'swear' : ['клясться', 'ругаться'], 'dream' : ['спать', 'мечтать']}

По ключу мы получим значение в виде списка:

In [48]:
my_dict['swear']

['клясться', 'ругаться']

Так как значением является список, можем отдельно обращаться к его элементам:

In [49]:
my_dict['swear'][0] # первый элемент

'клясться'

Можем пойти дальше и создать словарь, где значениями являются словари! Например, представим, что в некотором сообществе проходят выборы, и каждый участник может проголосовать за любое число кандидатов. Данные сохраняются в виде словаря, где ключами являются имена пользователей, а значениями – пары *кандидат-голос*.

In [50]:
votes = {'user1': {'cand1': '+', 'cand2': '-'},
         'user2' : {'cand1': 0, 'cand3' : '+'}} # '+' - за, '-' - против, 0 - воздержался

In [51]:
votes

{'user1': {'cand1': '+', 'cand2': '-'}, 'user2': {'cand1': 0, 'cand3': '+'}}

По аналогии с вложенными списками по ключам мы сможем обратиться к значению в словаре, который сам является значением в `votes` (да, эту фразу нужно осмыслить):

In [52]:
votes['user1']['cand1'] # берем значение, соответствующее ключу user1, в нем – ключу cand1

'+'

"Объединять словари можно через метод update(). Обратите внимание, что если в словарях есть одинаковые ключи, то они перезапишутся ключами того словаря, который добавляем."

In [53]:
votes.update(my_dict)
votes

{'user1': {'cand1': '+', 'cand2': '-'},
 'user2': {'cand1': 0, 'cand3': '+'},
 'swear': ['клясться', 'ругаться'],
 'dream': ['спать', 'мечтать']}

In [54]:
votes['swear'] = 1 # добавим в словарь votes ключ swear, чтобы проверить, что произойдет, когда объединим с my_dict, в котором есть такой ключ\n",
votes.update(my_dict)
print(votes) # swear в votes перезаписался swear из mydict

{'user1': {'cand1': '+', 'cand2': '-'}, 'user2': {'cand1': 0, 'cand3': '+'}, 'swear': ['клясться', 'ругаться'], 'dream': ['спать', 'мечтать']}


## Задача 4: слова

Напишите функцию `stats(s)`, принимающую на вход строку `s`, содержащую слова, разделённые пробелами, и находящую самое часто встречающееся слово. Если такое слово одно, верните его, если их несколько, верните список, отсортированный в лексикографическом порядке.

Например: `stats("hello hello world")` должна вернуть строчку `"hello"`, а `stats("a a b b c")` должна вернуть список `['a','b']`.

In [55]:
def stats(s):
    
    ### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
    # will the code be with you
    
        return

In [56]:
def test_problem(func, test_data):
    for inputs, true_answer in test_data:
        answer = func(inputs)
        assert answer == true_answer, f'Expected {true_answer}, got {answer}. Input: {inputs}'
    print("OK!")

In [ ]:
STATS_TESTS_DATA = [
        ("hello hello world", "hello"),
        ("a a b b c", ['a','b'])
    ]
test_problem(stats, STATS_TESTS_DATA)

## Задача 5: сумма двух 

Дан массив из целых чисел `nums` и ещё одно целое число `target`.  Найдите все такие пары чисел из массива `nums`, которые в сумме дают число `target`. Выведите на экран их индексы. Одно и то же число использовать при подсчёте суммы дважды нельзя. Попытайтесь решить эту задачу максимально эффективно. 

In [15]:
nums = [10, 20, 30, 40, 50]
target = 40

In [16]:
for i in range(len(nums)):
    for j in range(i + 1, len(nums)):
        if nums[i] + nums[j] == target :
            print(i, j)

0 2


N = 3, O = 9
N = 4, O = 16
N = 10_000, O = 10_000**2
O = N**2

In [19]:
num2index = {}
for i, num in enumerate(nums):
    num2index[num] = i

In [20]:
num2index

{10: 0, 20: 1, 30: 2, 40: 3, 50: 4}

In [22]:
for i, num in enumerate(nums):
    need_num = target - num
    if need_num in num2index:
        print(i, num2index[need_num])
        break

0 2


In [ ]:
def two_sum_fast(nums, target):
    
    # ┬─┬ ノ( ゜-゜ノ)
    
    # (╯° □°)╯︵ ┻━┻
    
    return ...

In [ ]:
TWO_SUM_TESTS_DATA = [
    (([2, 7, 11, 15], 9), [(0, 1)]),
]

test_problem_13(two_sum_fast, TWO_SUM_TESTS_DATA)

In [51]:
file = open('../sem06_dict/input.txt', 'r', encoding='utf-8')

for line in file:
    print(line.strip())

file.close()

Иванов Сергей 9 90
Сергеев Петр 10 91
Петров Василий 11 92
Васильев Иван 9 93


In [50]:
with open('../sem06_dict/input.txt', 'r', encoding='utf-8') as f:
    for line in f:
        print(line.strip())
# ...

Иванов Сергей 9 90
Сергеев Петр 10 91
Петров Василий 11 92
Васильев Иван 9 93


In [53]:
last_name, first_name, class_, ball = "Иванов Сергей 9 90".split()

In [54]:
?sorted

In [55]:
sorted([0, -20, 10, 66, 2, -4])

[-20, -4, 0, 2, 10, 66]

In [56]:
sorted([0, -20, 10, 66, 2, -4], reverse=True)

[66, 10, 2, 0, -4, -20]

In [57]:
l = [
    ["ahmed", 10],
    ["max", -5],
    ["anna", 5],
    ["tanya", 3],
]

In [64]:
d = {'a': 5, 'b': 3}

In [66]:
d.items()

dict_items([('a', 5), ('b', 3)])

In [68]:
sorted(d.items(), key=lambda x: x[1])

[('b', 3), ('a', 5)]

In [62]:
sorted(l, key=lambda x: x[1])

[['max', -5], ['tanya', 3], ['anna', 5], ['ahmed', 10]]

In [61]:
func = lambda x: x**2
func(5)

25

In [69]:
from collections import defaultdict

In [75]:
d = defaultdict(int)

In [76]:
d['a']

0

In [77]:
d = defaultdict(str)

In [78]:
d['a']

''

In [79]:
d = defaultdict(list)

In [80]:
d['a']

[]

In [86]:
d = defaultdict(lambda: defaultdict(int))

In [87]:
d['a']['b']

0

{'a': defaultdict(int, {'b': 0})}